In [3]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# 1. Carregar el dataset original
file_path = 'barcelona_compra_idealista.csv'
data = pd.read_csv(file_path)

# 2. Filtrar les dades
df_train = data.dropna(subset=['tags']).copy()  # Dades amb 'tags' existents
df_missing = data[data['tags'].isna()].copy()  # Dades amb 'tags' buits

# 3. Configurar el model BERT multilingüe
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')

# Funció per generar embeddings BERT
def generate_bert_embeddings(text_list):
    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=32)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embeddings

# 4. Generar embeddings per entrenament
X_train = generate_bert_embeddings(df_train['tags'].tolist())
y_train = df_train['tags'].tolist()

# Entrenar el model KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# 5. Generar embeddings per files sense 'tags'
if 'description' in data.columns:
    X_missing = generate_bert_embeddings(df_missing['description'].fillna("empty").tolist())
else:
    X_missing = generate_bert_embeddings(["unknown"] * len(df_missing))

# Predir els tags
predicted_tags = knn.predict(X_missing)

# 6. Assignar els valors predits
data.loc[data['tags'].isna(), 'tags'] = predicted_tags

# 7. Guardar el dataset actualitzat
data.to_csv('barcelona_compra_idealista_tags.csv', index=False)
print("Arxiu guardat com 'barcelona_compra_idealista_tags.csv'")


Arxiu guardat com 'barcelona_compra_idealista_tags.csv'


### Passos seguits per completar el procés d'assignació de valors a la columna 'tags' utilitzant embeddings BERT multilingües:

1. **Càrrega del dataset original**  
   Hem carregat el dataset `barcelona_compra_idealista.csv` per treballar amb les dades i identificar aquelles files que tenien valors buits a la columna 'tags'.

2. **Filtrat de les dades**  
   Hem dividit el dataset en dues parts:  
   - Dades que ja contenien un valor assignat a la columna 'tags', que hem utilitzat com a conjunt d'entrenament (`df_train`).  
   - Dades amb valors buits a la columna 'tags', que necessitaven ser omplertes (`df_missing`).

3. **Configuració del model BERT multilingüe**  
   Hem utilitzat el model `bert-base-multilingual-cased` de Hugging Face, que suporta diversos idiomes, ja que tenim text tant en català com en castellà.
   - Hem inicialitzat el **tokenizer** per convertir el text en entrades processables pel model.  
   - Hem carregat el **model preentrenat** per generar embeddings de les descripcions.

4. **Generació d'embeddings per al conjunt d'entrenament**  
   Hem creat una funció per generar embeddings a partir de text utilitzant el model BERT.  
   - Hem aplicat aquesta funció al text present a la columna 'tags' del conjunt d'entrenament (`df_train`) per obtenir representacions vectorials.  
   - Aquests embeddings s'han utilitzat com a conjunt de característiques (`X_train`), mentre que els valors originals de 'tags' s'han utilitzat com a etiquetes (`y_train`).

5. **Entrenament del model KNN**  
   Hem entrenat un classificador KNN (K-Nearest Neighbors) amb els embeddings generats i les etiquetes associades.  
   - Hem seleccionat `n_neighbors=3` per definir el nombre de veïns a considerar en les prediccions.

6. **Generació d'embeddings per a les files amb valors buits**  
   Hem aplicat la mateixa funció d'embeddings a la columna 'description' del conjunt de dades sense valors a 'tags'.  
   - Per evitar errors, hem substituït els valors buits de la columna 'description' per textos per defecte, com ara "empty" o "unknown".

7. **Predicció dels valors de 'tags' per a les files amb valors buits**  
   Amb el model KNN entrenat, hem predit els valors més probables de la columna 'tags' per a les files sense valor assignat (`df_missing`).

8. **Assignació dels valors predits i guardat del dataset**  
   Hem assignat els valors predits de 'tags' a les files corresponents del dataset original i hem guardat el dataset actualitzat en un fitxer CSV anomenat `barcelona_compra_idealista_tags.csv`.